In [11]:
# Imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from openpyxl import load_workbook
import time
import os
from typing import Tuple, Dict
import warnings

warnings.filterwarnings("ignore")


# ============================================================================
# CONFIGURATION
# ============================================================================

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

DEFAULT_CONFIG = {
    "hidden_size": 128,
    "num_layers": 2,
    "dropout": 0.1,
    "lr": 0.001,
    "lookback": 10,
}

BATCH_SIZE = 32
MAX_EPOCHS = 200
PATIENCE = 15
NUM_RUNS = 5


# ============================================================================
# REPRODUCIBILITY
# ============================================================================


def set_seed(seed=42):
    """Set all random seeds for reproducibility"""
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False


def seed_worker(worker_id):
    """Seed function for DataLoader workers"""
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)


# ============================================================================
# METRICS
# ============================================================================


def calculate_metrics(y_true: np.ndarray, y_pred: np.ndarray) -> Dict[str, float]:
    """Calculate MSE, RMSE, and R²"""
    mse = np.mean((y_true - y_pred) ** 2)
    rmse = np.sqrt(mse)

    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    r2 = 1 - (ss_res / ss_tot) if ss_tot != 0 else 0.0

    return {"mse": mse, "rmse": rmse, "r2": r2}


# ============================================================================
# DATA LOADING
# ============================================================================


def load_lstm_data_temporal(
    dataset_name: str, excel_path: str = "./datasets/data.xlsx"
) -> Tuple[np.ndarray, np.ndarray]:
    """Load time series data preserving temporal order from Excel"""

    wb = load_workbook(excel_path, read_only=True, data_only=True)

    if dataset_name not in wb.sheetnames:
        raise ValueError(f"Dataset '{dataset_name}' not found in workbook")

    ws = wb[dataset_name]

    # Read header
    header_row = next(ws.iter_rows(min_row=1, max_row=1, values_only=True))
    header = [
        str(cell).strip().upper() if cell is not None else f"COL_{i}"
        for i, cell in enumerate(header_row)
    ]

    # Find SERIE column
    serie_col = None
    for i, col_name in enumerate(header):
        if "SERIE" in col_name:
            serie_col = i
            break

    # Find SPLIT column
    split_col = None
    for i, col_name in enumerate(header):
        if "SPLIT" in col_name or "CONJUNTO" in col_name:
            split_col = i
            break

    if serie_col is None or split_col is None:
        raise ValueError(
            f"Could not identify SERIE and SPLIT columns for {dataset_name}"
        )

    print(
        f"[{dataset_name}] Using columns - SERIE: idx={serie_col} ({header[serie_col]}), SPLIT: idx={split_col} ({header[split_col]})"
    )

    # Read data and show first few rows for verification
    data = []
    for row_idx, row in enumerate(ws.iter_rows(min_row=2, values_only=True)):
        if row[serie_col] is not None:
            data.append(row)
            # Print first 3 data rows to verify correct columns
            if row_idx < 3:
                print(
                    f"  Row {row_idx+2}: SERIE={row[serie_col]}, SPLIT={row[split_col]}"
                )

    if len(data) == 0:
        raise ValueError(f"No data found in sheet '{dataset_name}'")

    df = pd.DataFrame(data)

    # Extract series and split labels
    series_data = pd.to_numeric(df.iloc[:, serie_col], errors="coerce").values
    split_labels = df.iloc[:, split_col].astype(str).str.strip().values

    # Remove NaN values
    valid_mask = ~np.isnan(series_data)
    series_data = series_data[valid_mask]
    split_labels = split_labels[valid_mask]

    # Normalize split labels
    split_labels_normalized = []
    for label in split_labels:
        label_lower = label.lower()
        if "treinamento" in label_lower:
            split_labels_normalized.append("Treinamento")
        elif "valida" in label_lower:
            split_labels_normalized.append("Validacao")
        elif "teste" in label_lower:
            split_labels_normalized.append("Teste")
        else:
            raise ValueError(f"Unknown split label: {label}")

    split_labels = np.array(split_labels_normalized)

    # Log data statistics for debugging
    print(
        f"[{dataset_name}] Data loaded: n={len(series_data)}, range=[{series_data.min():.2e}, {series_data.max():.2e}], mean={series_data.mean():.2e}"
    )
    print(
        f"[{dataset_name}] Splits: Train={np.sum(split_labels=='Treinamento')}, Val={np.sum(split_labels=='Validacao')}, Test={np.sum(split_labels=='Teste')}"
    )

    # Check for suspicious outliers (values > 10 when data should be in [0,1])
    # Remove corrupted values that are clearly data entry errors
    outlier_threshold = (
        10.0  # If normalized data should be [0,1], anything > 10 is corrupted
    )
    outlier_mask = series_data > outlier_threshold

    if np.any(outlier_mask):
        n_outliers = np.sum(outlier_mask)
        outlier_indices = np.where(outlier_mask)[0]
        print(
            f"[{dataset_name}] WARNING: Found {n_outliers} corrupted values > {outlier_threshold}"
        )
        print(f"  Outlier indices: {outlier_indices[:10]}")
        print(f"  Outlier values: {series_data[outlier_mask][:10]}")

        # Replace outliers with the median of surrounding valid values
        for idx in outlier_indices:
            # Get surrounding values (before and after)
            before_idx = max(0, idx - 5)
            after_idx = min(len(series_data), idx + 6)
            surrounding = series_data[before_idx:after_idx]
            valid_surrounding = surrounding[surrounding <= outlier_threshold]

            if len(valid_surrounding) > 0:
                replacement = np.median(valid_surrounding)
                series_data[idx] = replacement
                print(
                    f"  Replaced index {idx} (value={series_data[idx]:.2e}) with median={replacement:.4f}"
                )
            else:
                # Fallback: use overall median of series
                series_data[idx] = np.median(
                    series_data[series_data <= outlier_threshold]
                )

        print(
            f"[{dataset_name}] Fixed! New range: [{series_data.min():.2e}, {series_data.max():.2e}]"
        )

    return series_data, split_labels


# ============================================================================
# SEQUENCE CREATION
# ============================================================================


def create_sequences_no_leakage(
    series_data: np.ndarray, split_labels: np.ndarray, lookback: int
) -> Tuple:
    """
    Create sequences without data leakage across splits
    - Scaler fits only on training data
    - Sequences cannot look back across split boundaries
    """

    # Fit scaler only on training data
    train_mask = split_labels == "Treinamento"
    train_data = series_data[train_mask]

    if len(train_data) < 10:
        raise ValueError(f"Insufficient training data: {len(train_data)} points")

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler.fit(train_data.reshape(-1, 1))

    # Normalize entire series using training statistics
    series_normalized = scaler.transform(series_data.reshape(-1, 1)).ravel()

    # Create sequences
    X_train, y_train, y_train_orig = [], [], []
    X_val, y_val, y_val_orig = [], [], []
    X_test, y_test, y_test_orig = [], [], []

    for i in range(lookback, len(series_data)):
        sequence_norm = series_normalized[i - lookback : i]
        target_norm = series_normalized[i]
        target_orig = series_data[i]
        target_split = split_labels[i]

        lookback_splits = split_labels[i - lookback : i]

        # Only add if entire sequence (lookback + target) is from same split
        if target_split == "Treinamento" and np.all(lookback_splits == "Treinamento"):
            X_train.append(sequence_norm)
            y_train.append(target_norm)
            y_train_orig.append(target_orig)

        elif target_split == "Validacao" and np.all(lookback_splits == "Validacao"):
            X_val.append(sequence_norm)
            y_val.append(target_norm)
            y_val_orig.append(target_orig)

        elif target_split == "Teste" and np.all(lookback_splits == "Teste"):
            X_test.append(sequence_norm)
            y_test.append(target_norm)
            y_test_orig.append(target_orig)

    # Convert to numpy arrays
    X_train = (
        np.array(X_train) if len(X_train) > 0 else np.array([]).reshape(0, lookback)
    )
    y_train = np.array(y_train) if len(y_train) > 0 else np.array([])
    y_train_orig = np.array(y_train_orig) if len(y_train_orig) > 0 else np.array([])

    X_val = np.array(X_val) if len(X_val) > 0 else np.array([]).reshape(0, lookback)
    y_val = np.array(y_val) if len(y_val) > 0 else np.array([])
    y_val_orig = np.array(y_val_orig) if len(y_val_orig) > 0 else np.array([])

    X_test = np.array(X_test) if len(X_test) > 0 else np.array([]).reshape(0, lookback)
    y_test = np.array(y_test) if len(y_test) > 0 else np.array([])
    y_test_orig = np.array(y_test_orig) if len(y_test_orig) > 0 else np.array([])

    if len(X_train) < 10:
        raise ValueError(f"Insufficient training sequences: {len(X_train)}")

    return (
        X_train,
        y_train,
        X_val,
        y_val,
        X_test,
        y_test,
        y_train_orig,
        y_val_orig,
        y_test_orig,
        scaler,
    )


# ============================================================================
# DATASET CLASS
# ============================================================================


class TimeSeriesDataset(Dataset):
    """PyTorch Dataset for time series"""

    def __init__(self, X: np.ndarray, y: np.ndarray):
        self.X = torch.FloatTensor(X)
        self.y = torch.FloatTensor(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


# ============================================================================
# LSTM MODEL
# ============================================================================


class LSTMModel(nn.Module):
    """LSTM for time series forecasting"""

    def __init__(
        self,
        input_size: int = 1,
        hidden_size: int = 64,
        num_layers: int = 1,
        dropout: float = 0.0,
    ):
        super(LSTMModel, self).__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0,
        )

        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, (h, c) = self.lstm(x)
        out = out[:, -1, :]
        out = self.fc(out)
        return out.squeeze(-1)


# ============================================================================
# TRAINING
# ============================================================================


class EarlyStopping:
    """Early stopping handler"""

    def __init__(self, patience: int = 10, min_delta: float = 0.0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = float("inf")
        self.best_state = None
        self.best_epoch = 0

    def __call__(self, val_loss: float, model: nn.Module, epoch: int) -> bool:
        """Returns True if should stop training"""
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.best_state = model.state_dict().copy()
            self.best_epoch = epoch
            self.counter = 0
            return False
        else:
            self.counter += 1
            return self.counter >= self.patience

    def load_best_state(self, model: nn.Module):
        """Load best model state"""
        if self.best_state is not None:
            model.load_state_dict(self.best_state)


def train_one_epoch(
    model: nn.Module,
    train_loader: DataLoader,
    criterion: nn.Module,
    optimizer: torch.optim.Optimizer,
) -> float:
    """Train for one epoch"""
    model.train()
    total_loss = 0.0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        total_loss += loss.item() * len(X_batch)

    return total_loss / len(train_loader.dataset)


def validate_one_epoch(
    model: nn.Module, val_loader: DataLoader, criterion: nn.Module
) -> float:
    """Validate for one epoch"""
    model.eval()
    total_loss = 0.0

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            total_loss += loss.item() * len(X_batch)

    return total_loss / len(val_loader.dataset)


def train_lstm(
    train_loader: DataLoader,
    val_loader: DataLoader,
    config: Dict,
    max_epochs: int = 200,
    patience: int = 15,
) -> Tuple[nn.Module, Dict]:
    """Train LSTM with early stopping"""
    model = LSTMModel(
        input_size=1,
        hidden_size=config["hidden_size"],
        num_layers=config["num_layers"],
        dropout=config["dropout"],
    ).to(device)

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config["lr"])
    early_stopping = EarlyStopping(patience=patience)

    history = {"train_loss": [], "val_loss": [], "best_epoch": 0}

    for epoch in range(max_epochs):
        train_loss = train_one_epoch(model, train_loader, criterion, optimizer)
        val_loss = validate_one_epoch(model, val_loader, criterion)

        history["train_loss"].append(train_loss)
        history["val_loss"].append(val_loss)

        if early_stopping(val_loss, model, epoch + 1):
            break

    early_stopping.load_best_state(model)
    history["best_epoch"] = early_stopping.best_epoch

    return model, history


# ============================================================================
# EVALUATION
# ============================================================================


def evaluate_model(
    model: nn.Module, data_loader: DataLoader
) -> Tuple[np.ndarray, np.ndarray]:
    """Evaluate model and return predictions"""
    model.eval()
    y_true_list, y_pred_list = [], []

    with torch.no_grad():
        for X_batch, y_batch in data_loader:
            X_batch = X_batch.to(device)
            y_pred = model(X_batch)

            y_true_list.append(y_batch.cpu().numpy())
            y_pred_list.append(y_pred.cpu().numpy())

    return np.concatenate(y_true_list), np.concatenate(y_pred_list)


# ============================================================================
# RESULTS SAVING
# ============================================================================


def save_results_to_csv(results_data: Dict, filename: str = "results/lstm_results.csv"):
    """Save results to CSV"""
    os.makedirs(
        os.path.dirname(filename) if os.path.dirname(filename) else ".", exist_ok=True
    )

    df = pd.DataFrame([results_data])

    if os.path.exists(filename):
        df.to_csv(filename, mode="a", header=False, index=False)
    else:
        df.to_csv(filename, mode="w", header=True, index=False)


# ============================================================================
# MAIN EXPERIMENT PIPELINE
# ============================================================================


def run_single_experiment(
    dataset_name: str,
    run_num: int,
    config: Dict,
    series_data: np.ndarray,
    split_labels: np.ndarray,
) -> Dict:
    """Run single experiment"""

    set_seed(42 + run_num)
    # Always use DEFAULT_CONFIG for the model config
    config = DEFAULT_CONFIG
    lookback = config["lookback"]

    # Create sequences
    (
        X_train,
        y_train,
        X_val,
        y_val,
        X_test,
        y_test,
        y_train_orig,
        y_val_orig,
        y_test_orig,
        scaler,
    ) = create_sequences_no_leakage(series_data, split_labels, lookback)

    # Prepare data for LSTM
    X_train_reshaped = X_train.reshape(-1, lookback, 1)
    X_val_reshaped = X_val.reshape(-1, lookback, 1)
    X_test_reshaped = X_test.reshape(-1, lookback, 1)

    train_dataset = TimeSeriesDataset(X_train_reshaped, y_train)
    val_dataset = TimeSeriesDataset(X_val_reshaped, y_val)
    test_dataset = TimeSeriesDataset(X_test_reshaped, y_test)

    g = torch.Generator()
    g.manual_seed(42 + run_num)

    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        worker_init_fn=seed_worker,
        generator=g,
    )
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    train_loader_eval = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)

    # Train model
    start_time = time.time()
    model, history = train_lstm(
        train_loader, val_loader, config, max_epochs=MAX_EPOCHS, patience=PATIENCE
    )
    training_time = time.time() - start_time

    # Get predictions and denormalize
    _, y_pred_train_norm = evaluate_model(model, train_loader_eval)
    _, y_pred_val_norm = evaluate_model(model, val_loader)
    _, y_pred_test_norm = evaluate_model(model, test_loader)

    y_pred_train = scaler.inverse_transform(y_pred_train_norm.reshape(-1, 1)).ravel()
    y_pred_val = scaler.inverse_transform(y_pred_val_norm.reshape(-1, 1)).ravel()
    y_pred_test = scaler.inverse_transform(y_pred_test_norm.reshape(-1, 1)).ravel()

    # Calculate metrics on original scale
    train_metrics = calculate_metrics(y_train_orig, y_pred_train)
    val_metrics = calculate_metrics(y_val_orig, y_pred_val)
    test_metrics = calculate_metrics(y_test_orig, y_pred_test)

    # Prepare results
    results = {
        "experiment_num": run_num,
        "dataset": dataset_name,
        "method": "LSTM",
        "lookback": lookback,
        "hidden_size": config["hidden_size"],
        "num_layers": config["num_layers"],
        "dropout": config["dropout"],
        "learning_rate": config["lr"],
        "best_epoch": history["best_epoch"],
        "training_time_sec": training_time,
        "mse_train": train_metrics["mse"],
        "rmse_train": train_metrics["rmse"],
        "r2_train": train_metrics["r2"],
        "mse_val": val_metrics["mse"],
        "rmse_val": val_metrics["rmse"],
        "r2_val": val_metrics["r2"],
        "mse_test": test_metrics["mse"],
        "rmse_test": test_metrics["rmse"],
        "r2_test": test_metrics["r2"],
    }

    return results


def run_dataset_experiments(dataset_name: str):
    """Run all experiments for a single dataset"""

    # Always use DEFAULT_CONFIG for the model config
    config = DEFAULT_CONFIG

    # Load data
    series_data, split_labels = load_lstm_data_temporal(dataset_name)

    # Run multiple experiments
    dataset_results = []

    for run_num in range(1, NUM_RUNS + 1):
        try:
            results = run_single_experiment(
                dataset_name, run_num, config, series_data, split_labels
            )
            dataset_results.append(results)
            save_results_to_csv(results)
            print(
                f"{dataset_name} - Run {run_num}/{NUM_RUNS} - Test MSE: {results['mse_test']:.6e}, RMSE: {results['rmse_test']:.6e}"
            )

        except Exception as e:
            print(f"{dataset_name} - Run {run_num} failed: {e}")
            continue

    # Summary
    if dataset_results:
        test_mses = [r["mse_test"] for r in dataset_results]
        test_rmses = [r["rmse_test"] for r in dataset_results]
        print(f"\n{dataset_name} Summary:")
        print(f"Test MSE:  {np.mean(test_mses):.6e} ± {np.std(test_mses):.6e}")
        print(f"Test RMSE: {np.mean(test_rmses):.6e} ± {np.std(test_rmses):.6e}\n")

    return dataset_results


# ============================================================================
# MAIN EXECUTION
# ============================================================================

if __name__ == "__main__":
    DATASETS = [
        "CARSALES",
        "Electricity",
        "GAS",
        "LAKEERIE",
        "Nordic",
        "PIGS",
        "POLLUTION",
        "REDWINE",
        "SUNSPOT",
        "B1H",
    ]

    print("=" * 60)
    print("Starting LSTM Experiments")
    print(f"Datasets: {len(DATASETS)}")
    print(f"Runs per dataset: {NUM_RUNS}")
    print(f"Max epochs: {MAX_EPOCHS} (patience={PATIENCE})")
    print("=" * 60 + "\n")

    all_results = []

    for dataset_name in DATASETS:
        print(f"\n{'#'*60}")
        print(f"DATASET: {dataset_name}")
        print(f"{'#'*60}")

        results = run_dataset_experiments(dataset_name)
        all_results.extend(results)

    print(f"\n{'#'*60}")
    print(f"ALL EXPERIMENTS COMPLETED!")
    print(f"Total successful runs: {len(all_results)}")
    print(f"Results saved to: results/lstm_results.csv")
    print(f"{'#'*60}")

Starting LSTM Experiments
Datasets: 10
Runs per dataset: 5
Max epochs: 200 (patience=15)


############################################################
DATASET: CARSALES
############################################################
[CARSALES] Using columns - SERIE: idx=1 (SERIE), SPLIT: idx=2 (SPLIT)
  Row 2: SERIE=0.0478301105645122, SPLIT=Treinamento
  Row 3: SERIE=0.15391359407724903, SPLIT=Treinamento
  Row 4: SERIE=0.3145487311869855, SPLIT=Treinamento
[CARSALES] Data loaded: n=108, range=[0.00e+00, 1.00e+00], mean=4.40e-01
[CARSALES] Splits: Train=52, Val=36, Test=20
CARSALES - Run 1/5 - Test MSE: 1.224706e-01, RMSE: 3.499579e-01
CARSALES - Run 2/5 - Test MSE: 1.374454e-01, RMSE: 3.707362e-01
CARSALES - Run 3/5 - Test MSE: 1.559389e-01, RMSE: 3.948909e-01
CARSALES - Run 4/5 - Test MSE: 1.262397e-01, RMSE: 3.553023e-01
CARSALES - Run 5/5 - Test MSE: 1.390941e-01, RMSE: 3.729532e-01

CARSALES Summary:
Test MSE:  1.362377e-01 ± 1.172388e-02
Test RMSE: 3.687681e-01 ± 1.574130e-02


##